# TP 2 : récursivité et conditions

Dans tout le TP, il est interdit d'utiliser des boucles (pas de `for` ni `while`).

## Petites questions

1. Définir une fonction `divise` telle que `divise a b` renvoie `true` si  `a` divise `b`, `false` sinon. Il est interdit d'utiliser `if`.
1. Définir une fonction récursive `somme` telle que `somme n` renvoie $\sum_{k=1}^n k^2$.
2. Définir une fonction récursive `u` telle que `u n` renvoie $u_n$ définie par :
$$u_0 = 42$$
$$u_{n} = 3\sqrt{u_{n - 1}} + u_{n-2}$$
3. Soient $a$ et $b$ deux entiers et $q, r$ le quotient et reste de la division euclidienne de $a$ par $b$ ($a = bq + r$). En utilisant le fait que $PGCD(a, b)$ = $PGCD(b, r)$, écrire une fonction `euclide` telle que `euclide a b` renvoie le PGCD de $a$ et $b$.

In [1]:
(* Ex 1 *)
let divise a b =
    (b / a)*a = b
in
divise 3 9

- : bool = true


In [2]:
(* Ex 2 *)
let rec somme_sq n =
    if n = 1 then n
    else n*n+(somme_sq (n-1))
in
somme_sq 2

- : int = 5


In [3]:
(*Ex 3*)
let rec u n =
    if n = 0 then 42.
    else 3. *. (u (n-1))**0.5 +. 2.
in
u 3

- : float = 13.9593276159321622


In [4]:
(* Ex 4 *)
let rec euclide a b = 
    if b = 0 then a
    else 
    let r = a mod b in
    euclide b r
in
euclide 44 22

- : int = 22


## Exponentiation rapide

1. Écrire une fonction récursive `puissance` naïve (c'est à dire très simple) telle que `puissance a n` renvoie $a^n$. Combien effectue t-elle de multiplications (en fonction de n) ?
2. Écrire une fonction récursive `exp_rapide` pour calculer $a^n$, en utilisant les relations suivantes :
$$ 
\begin{cases} 
a^n = (a^{\frac{n}{2}})^2 ~~~~~~~~~\text{si }n\text{ est pair}\\
a^{n} = a \times (a^{\frac{n-1}{2}})^2 ~~~~~\text{sinon}
\end{cases}
$$
**Remarque** : On montrera plus tard que cette 2ème version demande environ $\ln(n)$ multiplications seulement.

In [5]:
let rec puissance_nai_entier a n =
    if n = 0 then 1
    else a*puissance_nai_entier a (n-1)
;;
puissance_nai_entier 2 3

val puissance_nai_entier : int -> int -> int = <fun>


- : int = 8


In [6]:
let rec puissance_entier a n =
    if n = 1 then a 
    else if n mod 2 = 0 
        then puissance_nai_entier (puissance_entier a (n/2)) 2
        else a*puissance_nai_entier (puissance_entier a ((n-1)/2)) 2
in
puissance_entier 2 20


- : int = 1048576


# Accumulateur

On a vu dans le cours sur la récursivité (avec l'exemple de la suite de Fibonacci) qu'un accumulateur est un argument que l'on ajoute à une fonction pour calculer sa valeur de retour.  
1. Écrire une fonction `fact` telle que `fact acc n` renvoie `n`!, en utilisant `acc` comme accumulateur. Voici à quoi va ressemble `fact` :
```ocaml
let rec fact acc n =
    if n = 0 then acc  (* on renvoie l'accumulateur qui contient le résultat *)
    else fact ... (* appel récursif en modifiant l'accumulateur *)
```
2. En utilisant `fact` et l'application partielle de fonction, définir `f : int -> int` renvoyant la factoielle d'un entier.

**Remarque** : le but ici est juste de vous entraîner à savoir utiliser un accumulateur, qui sont parfois utiles (comme pour la fonction `fibo` du cours). En DS ou concours on évitera d'utiliser un accumulateur lorsqu'il y en a pas besoin (comme pour la fonction `fact`...), car cela rend le code plus compliqué.  

In [7]:
(*Ex 1 *)
let rec fact acc n =
    if n=0 then acc
    else (fact (acc*n) (n-1))
in
fact 1 4

- : int = 24


In [8]:
(*Ex 2 *)
let rec fact2 n =
    if n=0 then 1
    else n*fact2 (n-1)
;;

val fact2 : int -> int = <fun>


## Temps de vol de la suite de Syracuse

La suite de Syracuse d'un entier $a$ est définie par :  
$$u_0 = a$$
$$u_{n+1} =
\begin{cases} 
     \frac{u_n}{2}, \text{si } n \text{ est pair}\\
    3u_n + 1, \text{sinon}\\
\end{cases}$$

Le temps de vol de $(u_n)_n$ est le plus petit entier $t$ tel que $u_t = 1$.  

1. Écrire une fonction récursive `temps_vol` telle que `temps_vol a` renvoie le temps de vol de $(u_n)_n$ (où $u_0 = a$).

In [9]:
let rec temps_vol a =
    if a = 1 then 0
    else if a mod 2 = 0
    then 1+temps_vol (a/2)
    else 1+temps_vol (a*3+1)
in
temps_vol 8

- : int = 3


## Retour sur les tours de Hanoi

**Remarque** : le problème des tours de Hanoi a déjà été vu en stage, mais on le refait ici en OCaml. En outre, il est utile de réactiver sa mémoire pour se souvenir des méthodes et il m'arrivera pendant l'année de redonner des exercices déjà posés.

![](../img/hanoi.png)

$n$ disques sont posés sur la tige à gauche. L'objectif est de déplacer tous les disques sur la tige à droite :

![](../img/hanoi2.png)

Règles du jeu :

- On ne peut déplacer qu'un disque à la fois (celui tout en haut), sur une autre tige.
- Il est interdit de poser un disque sur un autre plus petit.

Exemple de premier déplacement valide :

![](../img/hanoi3.png)

On souhaite écrire une fonction récursive `hanoi` telle que `hanoi n tige1 tige2` affiche une suite de déplacements (avec des `print_int`) permettant de déplacer $n$ disques depuis `tige1` vers `tige2`. On supposera que les tiges sont numérotées 0, 1, 2 (de gauche à droite).

1. Supposons que vous sachiez déplacer $n-1$ disques d'une tige à une autre. Comment déplacer $n$ disques d'une tige à une autre ?
2. Écrire `hanoi`.

In [39]:
(*Exercice 1*)
(* On souhaiterait déplacer les n-1 disques sur la tige 1, déplacer le n-nième disque sur la tige 2, puis déplacer n-1 disque sur la tige 2 *)
(*Exercice 2*)
let rec hanoi n tige_origine tige_but =
    if n = 1 (*ici on fait le cas pour 1 un seul disque*)
    then 
        print_int tige_origine;
        print_int tige_but
    else if tige_but = 2 (*Je ne sais pas ce qui provoque l'erreur*)
    then (*ici on cherche à ammener les n disques à la tige 1*)
        hanoi (n-1) tige_origine (tige_but-1);
        print_int tige_origine;
        print_int tige_but;
        hanoi (n-1) (tige_but-1) tige_but
    else (*si on cherche à ammener les n disques à la tiges 1, alors on ammènera d'abord les n-1 disques à la tige 2*)
        hanoi (n-1) tige_origine (tige_but+1);
        print_int tige_origine;
        print_int tige_but;
        hanoi (n-1) (tige_but+1) tige_but
in
hanoi 3 0 2;

error: compile_error